## Projeto Modulo 3 & 4

In [ ]:
### Parte Técnica - Modulo 3 (Bibliotecas APIs)
### Parte Agile   - Modulo 4  (Seaborn)

### Preferência do Público - Atores

**DESCRIÇÃO**

Eu, como Gestor de conteúdo, gostaria de visualizar os atores com maior preferência do público, para cotá-los em futuras produções.

**REGRAS DE NEGÓCIO**
- Levar em consideração apenas atores VIVOS  &#x2705;
- Somente filmes devem ser considerados 	 &#x2705; 
- Apenas filmes lançados nos últimos 5 anos  &#x2705;
- Para notas genéricas (pelo filme), considerar apenas os atores principais &#x2705;

**CRITÉRIOS DE ACEITE**
- Quero receber uma lista com os 6 mais bem cotados &#x2705;
- Devem ser 3 homens e 3 mulheres 	 &#x2705;


#### Bibliotecas

In [1]:
from tqdm import tqdm, tqdm_notebook
from IPython.display import clear_output
import numpy as np
import pandas as pd
import os

#### Importando Base de Dados

In [2]:
# Importando todos os arquivos de uma pasta.

## Windows x Linux
#path = '/home/csvaz/dev/projeto/data'
path = 'C:\DataScience\projeto\data'

for _, _, arquivo in os.walk(path):
    print('Arquivos Disponiveis:')
    print(arquivo)

    
for arq in tqdm_notebook(arquivo, desc= 'Carregando...'):
    name = arq
    name = name.replace('IMDb ', '')
    name = name.replace('.csv', '')
    arq = '{}/{}'.format(path,arq)
       
    globals()['df_' + name] = pd.read_csv(arq)
    print('df_{}'.format(name))
    

    


Arquivos Disponiveis:
['IMDb movies.csv', 'IMDb names.csv', 'IMDb ratings.csv']


df_movies
df_names
df_ratings



##### Funções de Suporte

In [3]:
def define_gender(x):
    if 'actor' in str(x):
        return 'M'
    if 'actress' in str(x):
        return 'F'
    else:
        return np.nan
    


#### ETL Movies

In [4]:
df_movies.head(2)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,NaN,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,537,$ 2250,NaN,NaN,NaN,7.0,7.0
1,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.9,171,NaN,NaN,NaN,NaN,4.0,2.0


In [5]:
df_etl_movies = df_movies[['imdb_title_id', 'title', 'year', 'avg_vote']]
df_etl_movies.head(2)
df_etl_movies.year.unique()

array([1906, 1911, 1912, 1919, 1913, 1914, 1915, 1916, 1917, 1918, 1920,
       1921, 1924, 1922, 1923, 1925, 1926, 1935, 1927, 1928, 1929, 1930,
       1932, 1931, 1937, 1938, 1933, 1934, 1936, 1940, 1939, 1942, 1943,
       1941, 1948, 1944, 2001, 1946, 1945, 1983, 1947, 1973, 1949, 1950,
       1952, 1951, 1962, 1953, 1954, 1955, 1961, 1956, 1958, 1957, 1959,
       1960, 1963, 1965, 1971, 1964, 1966, 1967, 1969, 1976, 1968, 1979,
       1970, 1972, 1981, 1978, 1989, 1990, 1975, 1974, 1986, 2018, 1977,
       1982, 1980, 1993, 1984, 1985, 1988, 1987, 2019, 2005, 1991, 2002,
       1994, 1992, 1995, 2017, 1997, 1996, 1999, 1998, 2000, 2007, 2008,
       2003, 2004, 2010, 2009, 2011, 2006, 2012, 2013, 2016, 2015, 2014],
      dtype=int64)

In [6]:
df_etl_movies = df_etl_movies[df_etl_movies.year > 2015]

df_etl_movies.head(2)

,imdb_title_id,title,year,avg_vote
15090,tt0069049,The Other Side of the Wind,2018,6.9
20324,tt0085953,Mo tai,2019,6.2


#### ETL Names

In [7]:
df_etl = df_names[['imdb_name_id', 'name', 'date_of_birth', 'death_year', 'primary_profession', 'known_for_titles']]
pd.options.mode.chained_assignment = None # Desabilita o Warning
df_etl['Gender'] = df_etl['primary_profession'].apply(define_gender) # Define Sexo
df_etl = df_etl[(df_etl.death_year.isnull()) & (df_etl.Gender.notnull()) & (df_etl.Gender != 'None')] # Remove


In [8]:
df_split_titles = df_etl.known_for_titles.str.split(expand=True, pat=',')
df_split_titles.columns = ['Titulo1', 'Titulo2', 'Titulo3', 'Titulo4', 'Titulo5']
df_split_titles = pd.concat([df_etl, df_split_titles],axis = 1)
df_split_titles.head(5)

,imdb_name_id,name,date_of_birth,death_year,primary_profession,known_for_titles,Gender,Titulo1,Titulo2,Titulo3,Titulo4,Titulo5
2,nm0000003,Brigitte Bardot,1934-09-28,NaN,"actress,soundtrack,producer","tt0054452,tt0059956,tt0057345,tt0049189",F,tt0054452,tt0059956,tt0057345,tt0049189,None
13,nm0000014,Olivia de Havilland,1916-07-01,NaN,"actress,soundtrack","tt0041452,tt0031381,tt0040806,tt0029843",F,tt0041452,tt0031381,tt0040806,tt0029843,None
17,nm0000018,Kirk Douglas,1916-12-09,NaN,"actor,producer,soundtrack","tt0080736,tt0043338,tt0052365,tt0054331",M,tt0080736,tt0043338,tt0052365,tt0054331,None
46,nm0000047,Sophia Loren,1934-09-20,NaN,"actress,soundtrack","tt0054749,tt0058335,tt0060121,tt0076085",F,tt0054749,tt0058335,tt0060121,tt0076085,None
78,nm0000079,Raquel Welch,1940-09-05,NaN,"actress,soundtrack,producer","tt0072281,tt0061653,tt0068795,tt0070291",F,tt0072281,tt0061653,tt0068795,tt0070291,None


In [9]:
for i in df_split_titles.iloc[:,7:12]:
    
    
    df_split_titles = df_split_titles.merge(df_etl_movies, left_on=i, right_on='imdb_title_id',how = 'left')
    df_split_titles.drop('imdb_title_id', axis=1, inplace=True)
    df_split_titles.drop('title', axis=1, inplace=True)
    df_split_titles.drop('year', axis=1, inplace=True)
    df_split_titles.rename({'avg_vote':'Voto'}, axis=1, inplace=True)

df_split_titles.head(2)


,imdb_name_id,name,date_of_birth,death_year,primary_profession,known_for_titles,Gender,Titulo1,Titulo2,Titulo3,Titulo4,Titulo5,Voto,Voto,Voto,Voto,Voto
0,nm0000003,Brigitte Bardot,1934-09-28,NaN,"actress,soundtrack,producer","tt0054452,tt0059956,tt0057345,tt0049189",F,tt0054452,tt0059956,tt0057345,tt0049189,None,NaN,NaN,NaN,NaN,NaN
1,nm0000014,Olivia de Havilland,1916-07-01,NaN,"actress,soundtrack","tt0041452,tt0031381,tt0040806,tt0029843",F,tt0041452,tt0031381,tt0040806,tt0029843,None,NaN,NaN,NaN,NaN,NaN


In [10]:
df_split_titles['avg_total'] = round(df_split_titles.iloc[:,12:17].mean(axis = 1, skipna = True),2)
df_split_titles.head(2)

,imdb_name_id,name,date_of_birth,death_year,primary_profession,known_for_titles,Gender,Titulo1,Titulo2,Titulo3,Titulo4,Titulo5,Voto,Voto,Voto,Voto,Voto,avg_total
0,nm0000003,Brigitte Bardot,1934-09-28,NaN,"actress,soundtrack,producer","tt0054452,tt0059956,tt0057345,tt0049189",F,tt0054452,tt0059956,tt0057345,tt0049189,None,NaN,NaN,NaN,NaN,NaN,NaN
1,nm0000014,Olivia de Havilland,1916-07-01,NaN,"actress,soundtrack","tt0041452,tt0031381,tt0040806,tt0029843",F,tt0041452,tt0031381,tt0040806,tt0029843,None,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_etl = df_split_titles[['name','Gender','avg_total']]
df_etl = df_etl.sort_values(by='avg_total', ascending = False  )


### Resultado 

In [25]:
df_etl.head(6)

,name,Gender,avg_total
46601,Kathy Searle,F,10.0
77991,Sangeetha Bhat,F,9.6
71783,Aparna Gopinath,F,9.5
38161,Pradeep Sharma,M,9.5
38160,Preetyka Chauhan,M,9.5
38159,Pratap Saurabh,M,9.5
